# Task

#### Build a Convolutional Neural Network (CNN) model to classify images from a given dataset into predefined categories/classes.

## The Dataset

### We could choose between Animals 10 or 

#### The dataset for this task is the CIFAR-10 dataset

#### - 60,000 32x32 color images in 10 classes
#### - 6,000 images per class.

## Imports

In [1]:
import os
import pickle
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.utils import image_dataset_from_directory, to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation, RandomFlip, RandomTranslation, Lambda

2026-01-29 15:20:36.587395: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-29 15:20:36.660684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-29 15:20:36.695114: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-29 15:20:36.712220: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-29 15:20:36.777696: I tensorflow/core/platform/cpu_feature_guar

## Setting the random seed

In [2]:
seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

## Transfer Training VGG19

### Loading the Data Set Again and Using the preprocessig Libraries of the VGG19

In [6]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train_ohe = to_categorical(y_train, 10)
y_test_ohe  = to_categorical(y_test, 10)

X_train_pp = preprocess_input(X_train.astype("float32"))
X_test_pp  = preprocess_input(X_test.astype("float32"))

### Building the VGG19

In [27]:
vgg_transfer = VGG19(
    include_top=False,
    weights="imagenet",      
    input_shape=(32, 32, 3)
)

vgg_transfer.trainable = False    #This freezes

In [28]:
vgg_model = Sequential([ 
    RandomFlip("horizontal"),
    RandomTranslation(0.1, 0.1),
    
    Lambda(preprocess_input, name="vgg19_preprocess"),

    vgg_transfer,

    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(10, activation="softmax")    
])

vgg_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

#vgg_model.summary()

In [29]:
vgg_transfer.trainable = True   # Unfreezes

for layer in vgg_model.layers[:-4]:
    layer.trainable = False

vgg_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=2, min_lr=1e-4
)

In [30]:
vgg_history = vgg_model.fit(
    X_train, y_train_ohe,
    validation_split=0.1,
    epochs=50,
    batch_size=128,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/50


'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring f

  6/352 [..............................] - ETA: 9s - loss: 12.9264 - accuracy: 0.0964 

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)


352/352 [==============================] - 12s 24ms/step - loss: 2.3596 - accuracy: 0.1667 - val_loss: 1.9225 - val_accuracy: 0.2330 - lr: 0.0010
Epoch 2/50
352/352 [==============================] - 7s 21ms/step - loss: 1.9144 - accuracy: 0.2245 - val_loss: 1.7478 - val_accuracy: 0.2704 - lr: 0.0010
Epoch 3/50
352/352 [==============================] - 7s 20ms/step - loss: 1.7923 - accuracy: 0.2650 - val_loss: 1.6129 - val_accuracy: 0.3458 - lr: 0.0010
Epoch 4/50
352/352 [==============================] - 7s 20ms/step - loss: 1.6734 - accuracy: 0.3283 - val_loss: 1.5425 - val_accuracy: 0.4066 - lr: 0.0010
Epoch 5/50
352/352 [==============================] - 7s 20ms/step - loss: 1.5765 - accuracy: 0.3790 - val_loss: 1.4379 - val_accuracy: 0.4036 - lr: 0.0010
Epoch 6/50
352/352 [==============================] - 7s 20ms/step - loss: 1.3475 - accuracy: 0.4786 - val_loss: 1.1265 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 7/50
352/352 [==============================] - 7s 21ms/step - loss:

In [31]:
test_loss, test_acc= vgg_model.evaluate(X_test, y_test_ohe , verbose=1)
print("Test accuracy:", test_acc)
print("Test loss:", test_loss)
print("Test Top3 accuracy:")

W0000 00:00:1769705845.165546  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.168746  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.168962  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.178483  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.179574  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.181548  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.182894  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.183548  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.184470  222861 gp

 25/313 [=>............................] - ETA: 1s - loss: 0.4278 - accuracy: 0.8712

W0000 00:00:1769705845.367390  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.367576  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.367860  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.368245  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.369205  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.369580  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.369848  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.370207  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705845.370462  222861 gp

313/313 [==============================] - 3s 7ms/step - loss: 0.5028 - accuracy: 0.8523
Test accuracy: 0.8522999882698059
Test loss: 0.5027539730072021
Test Top3 accuracy:


W0000 00:00:1769705847.483954  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705847.484069  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705847.484209  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705847.484724  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705847.485039  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705847.485607  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705847.485989  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705847.486178  222861 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1769705847.486387  222861 gp